# PySpark DataFrame API & SQL — 45 Hands‑On Examples

This notebook contains **45 runnable examples** (15 per section) covering:

1) **Reading/Writing data** (CSV, JSON, Parquet)

2) **Data transformations & aggregations**

3) **Spark SQL querying**

Each example is structured as:

- *What the function does* (Markdown)

- *Code cell*

- *Explanation & optimization tips*

> **Note:** If running in Colab/locally, ensure PySpark is installed and a SparkSession is available.



## Prerequisites
If you're in **Colab**, run the first install cell. Locally, ensure Java and PySpark are installed.

- Python 3.8+
- Java 8/11/17
- PySpark 3.x


In [ ]:
# Optional (Colab): Install PySpark
# !pip install -q pyspark

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
spark = SparkSession.builder.appName("PySpark-DF-API-SQL-Examples").getOrCreate()

# Keep shuffle partitions small for demos
spark.conf.set("spark.sql.shuffle.partitions", "4")

print("Spark version:", spark.version)


## Demo Data Setup
We create small demo datasets and write them to local `data/` so read/write examples work anywhere.

In [ ]:
import os, shutil
base_dir = "data"
input_dir = os.path.join(base_dir, "input")
output_dir = os.path.join(base_dir, "output")
for d in [input_dir, output_dir]:
    if os.path.exists(d):
        shutil.rmtree(d)
    os.makedirs(d, exist_ok=True)

# Demo data
customers = [
    (1, "Alice", "2024-01-10", "Bangalore"),
    (2, "Bob",   "2024-02-20", "Pune"),
    (3, "Carol", "2024-02-25", "Hyderabad"),
    (4, "Dan",   "2024-03-15", "Bangalore"),
    (5, "Eve",   "2024-03-18", "Mumbai"),
]

products = [
    (101, "Keyboard", "peripheral", 999.0),
    (102, "Mouse", "peripheral", 499.0),
    (103, "Monitor", "display", 12499.0),
    (104, "Laptop", "computer", 73999.0),
    (105, "Dock", "peripheral", 3499.0),
]

sales = [
    (1, 101, 2, "2024-03-20"),
    (1, 102, 1, "2024-03-20"),
    (2, 104, 1, "2024-04-03"),
    (3, 103, 2, "2024-04-07"),
    (4, 101, 1, "2024-04-10"),
    (5, 105, 1, "2024-04-12"),
    (5, 103, 1, "2024-04-12"),
]

schema_customers = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("signup_date", StringType(), False),
    StructField("city", StringType(), True),
])

schema_products = StructType([
    StructField("product_id", IntegerType(), False),
    StructField("product", StringType(), False),
    StructField("category", StringType(), False),
    StructField("price", DoubleType(), False),
])

schema_sales = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("product_id", IntegerType(), False),
    StructField("qty", IntegerType(), False),
    StructField("sale_date", StringType(), False),
])

df_customers = spark.createDataFrame(customers, schema_customers)
df_products  = spark.createDataFrame(products, schema_products)
df_sales     = spark.createDataFrame(sales, schema_sales)

# Write demo inputs
df_customers.write.mode("overwrite").option("header", True).csv(os.path.join(input_dir, "customers_csv"))
df_products.write.mode("overwrite").json(os.path.join(input_dir, "products_json"))
df_sales.write.mode("overwrite").parquet(os.path.join(input_dir, "sales_parquet"))

print("Sample files created under:", input_dir)


## 1) Reading/Writing Data (CSV, JSON, Parquet) — 15 Examples

### Reading/Writing — Example 1: `spark.read.csv`

**What this is about:** Read CSV with header & inferSchema.

In [ ]:
path = "data/input/customers_csv"
df = spark.read.option("header", True).option("inferSchema", True).csv(path)
df.show()
df.printSchema()


**Explanation:**
We read a CSV directory with a header; Spark infers column types automatically.

**Optimization options:**
- Prefer **explicit schemas** in production for stability.
- Control parsing with `mode`, `timestampFormat`, etc.

### Reading/Writing — Example 2: `spark.read.csv + schema`

**What this is about:** Read CSV using an explicit `StructType` schema.

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("signup_date", StringType(), False),
    StructField("city", StringType(), True),
])
df = spark.read.option("header", True).schema(schema).csv("data/input/customers_csv")
df.show()


**Explanation:**
Explicit schema avoids surprises from inferSchema and speeds up planning.

**Optimization options:**
- Use schemas for **data contracts**.
- Validate with `dropMalformed`/`DROPMALFORMED` when needed.

### Reading/Writing — Example 3: `CSV options (sep, quote, escape, mode)`

**What this is about:** Control parsing with `sep`, `quote`, `escape`, and `mode`.

In [ ]:
df = spark.read.options(header=True, sep=",", quote=""", escape="\", mode="PERMISSIVE") \
    .csv("data/input/customers_csv")
df.show()


**Explanation:**
These options handle delimiters and quoted fields; `mode` controls malformed rows handling.

**Optimization options:**
- Use `DROPMALFORMED` or `FAILFAST` to enforce quality.
- Keep consistent delimiters upstream.

### Reading/Writing — Example 4: `DataFrame.write.csv`

**What this is about:** Write a DataFrame as CSV with header.

In [ ]:
out = "data/output/customers_csv_out"
(df_customers
 .coalesce(1)  # single file for tiny demo
 .write.mode("overwrite")
 .option("header", True)
 .csv(out))
print("Wrote to", out)


**Explanation:**
`coalesce(1)` forces a single output file (for small demos only).

**Optimization options:**
- Avoid single-file writes for big data.
- Use partitioned writes for scalability.

### Reading/Writing — Example 5: `partitionBy + write.csv`

**What this is about:** Partition CSV output by a column.

In [ ]:
out = "data/output/customers_by_city_csv"
(df_customers
 .repartition(2, "city")
 .write.mode("overwrite")
 .option("header", True)
 .partitionBy("city")
 .csv(out))
print("Partitioned output ->", out)


**Explanation:**
Partitions create folder structure by partition keys for efficient reads.

**Optimization options:**
- Partition by **low-cardinality** columns.
- Tune `repartition` to match file sizes.

### Reading/Writing — Example 6: `spark.read.json`

**What this is about:** Read JSON files from a directory.

In [ ]:
df = spark.read.json("data/input/products_json")
df.show()
df.printSchema()


**Explanation:**
Spark infers JSON structure; nested fields are supported.

**Optimization options:**
- For big JSON, consider **multiline** and schema evolution.

### Reading/Writing — Example 7: `multiline JSON`

**What this is about:** Enable `multiLine=True` for pretty-printed JSON.

In [ ]:
# Demo: write a multiline JSON file
ml_path = "data/input/products_json_multiline"
(df_products.coalesce(1)
 .write.mode("overwrite")
 .option("compression", "uncompressed")
 .json(ml_path))

df = spark.read.option("multiLine", True).json(ml_path)
df.show()


**Explanation:**
`multiLine` parses JSON objects spanning multiple lines.

**Optimization options:**
- Be careful: multiline parsing is slower.
- Prefer newline-delimited JSON for streaming/big data.

### Reading/Writing — Example 8: `DataFrame.write.json + compression`

**What this is about:** Write JSON with compression (e.g., gzip).

In [ ]:
out = "data/output/products_json_gz"
(df_products
 .write.mode("overwrite")
 .option("compression", "gzip")
 .json(out))
print("Compressed JSON ->", out)


**Explanation:**
Compressed JSON reduces storage/read I/O.

**Optimization options:**
- Use compressions supported by downstream tools.
- Gzip helps archiving; Snappy is CPU-friendly.

### Reading/Writing — Example 9: `spark.read.parquet`

**What this is about:** Read Parquet files (columnar, compressed).

In [ ]:
df = spark.read.parquet("data/input/sales_parquet")
df.show()
df.printSchema()


**Explanation:**
Parquet is the default choice for analytics: columnar + schema + compression.

**Optimization options:**
- Prefer Parquet for lakehouse analytics.
- Combine with partition pruning for speed.

### Reading/Writing — Example 10: `DataFrame.write.parquet + partition + compression`

**What this is about:** Write Parquet partitioned with Snappy compression.

In [ ]:
out = "data/output/sales_parquet_by_date"
(df_sales
 .withColumn("ym", F.substring("sale_date", 1, 7))
 .repartition(2, "ym")
 .write.mode("overwrite")
 .option("compression", "snappy")
 .partitionBy("ym")
 .parquet(out))
print("Parquet out ->", out)


**Explanation:**
Writes compressed partitioned Parquet for efficient analytics.

**Optimization options:**
- Choose partition column aligned to query filters (e.g., by month).

### Reading/Writing — Example 11: `DataFrameReader.load(format=...)`

**What this is about:** Use generic `.load()` with a format.

In [ ]:
df = (spark.read
      .format("csv")
      .option("header", True)
      .load("data/input/customers_csv"))
df.show()


**Explanation:**
`.load()` is unified across formats (csv/json/parquet).

**Optimization options:**
- Standardize with `.format()` for flexible pipelines.

### Reading/Writing — Example 12: `Read with DDL schema string`

**What this is about:** Provide schema via DDL string (fast & compact).

In [ ]:
ddl = "customer_id INT, name STRING, signup_date STRING, city STRING"
df = spark.read.option("header", True).schema(ddl).csv("data/input/customers_csv")
df.printSchema()


**Explanation:**
DDL strings save code and are easy to read.

**Optimization options:**
- Keep DDL in config files to decouple from code.

### Reading/Writing — Example 13: `repartition vs coalesce before write`

**What this is about:** Control the number of output files before writing.

In [ ]:
out1 = "data/output/sales_repartition_4"
(df_sales.repartition(4).write.mode("overwrite").parquet(out1))

out2 = "data/output/sales_coalesce_1"
(df_sales.coalesce(1).write.mode("overwrite").parquet(out2))

print("Wrote:", out1, "and", out2)


**Explanation:**
`repartition(N)` shuffles to increase partitions; `coalesce(N)` reduces without full shuffle.

**Optimization options:**
- Use `coalesce` to reduce small files cheaply.
- Use `repartition` to spread heavy workloads.

### Reading/Writing — Example 14: `saveAsTable (requires Hive support)`

**What this is about:** Save a DataFrame as a managed table (if Hive support configured).

In [ ]:
# This may require extra config (warehouse dir + Hive support)
# spark.sql("CREATE DATABASE IF NOT EXISTS demo_db")
# spark.catalog.setCurrentDatabase("demo_db")
# df_products.write.mode("overwrite").saveAsTable("products_tbl")
# spark.sql("SELECT * FROM products_tbl").show()
print("Note: saveAsTable requires Spark with Hive/metastore configured.")


**Explanation:**
Tables integrate with the metastore; great for BI and SQL UIs.

**Optimization options:**
- In local demos without Hive, prefer Parquet + views.

### Reading/Writing — Example 15: `Parquet mergeSchema`

**What this is about:** Read partitioned Parquet with schema evolution via `mergeSchema`.

In [ ]:
# Create a second parquet with an extra column to illustrate mergeSchema
extra = df_sales.withColumn("seller", F.lit("online"))
extra_dir = "data/input/sales_parquet_extra"
extra.write.mode("overwrite").parquet(extra_dir)

merged = spark.read.option("mergeSchema", True).parquet("data/input/sales_parquet", extra_dir)
merged.printSchema()


**Explanation:**
`mergeSchema` merges differing schemas across paths (more planning cost).

**Optimization options:**
- Use sparingly; prefer planned schema evolution.

## 2) Data Transformations & Aggregations — 15 Examples

### Transformations & Aggregations — Example 1: `select & alias`

**What this is about:** Select columns with aliases.

In [ ]:
df = df_products.select(F.col("product_id").alias("pid"),
                        F.col("product"),
                        F.col("category"),
                        F.col("price"))
df.show()


**Explanation:**
`select` projects columns; `alias` renames for readability.

**Optimization options:**
- Select only required columns to reduce I/O.

### Transformations & Aggregations — Example 2: `withColumn`

**What this is about:** Create or replace a column (e.g., price with tax).

In [ ]:
df = df_products.withColumn("price_with_gst", F.col("price") * 1.18)
df.show()


**Explanation:**
`withColumn` derives new columns; here we add 18% GST.

**Optimization options:**
- Chain transformations; avoid multiple scans by combining ops.

### Transformations & Aggregations — Example 3: `filter / where`

**What this is about:** Filter rows using expressions.

In [ ]:
df = df_products.filter(F.col("category") == "peripheral")
df.show()


**Explanation:**
`filter`/`where` keeps only matching rows.

**Optimization options:**
- Push filters down by reading only relevant partitions.

### Transformations & Aggregations — Example 4: `drop & dropDuplicates`

**What this is about:** Drop columns and remove duplicate rows by subset.

In [ ]:
df = df_customers.drop("signup_date").dropDuplicates(["city"])
df.show()


**Explanation:**
`drop` removes columns; `dropDuplicates` dedups by keys.

**Optimization options:**
- Dedup post-join with `dropDuplicates` using business keys.

### Transformations & Aggregations — Example 5: `groupBy().agg()`

**What this is about:** Aggregate totals by category.

In [ ]:
agg = (df_products.groupBy("category")
       .agg(F.count("*").alias("cnt"),
            F.avg("price").alias("avg_price"),
            F.max("price").alias("max_price")))
agg.show()


**Explanation:**
Group aggregations compute metrics by groups.

**Optimization options:**
- Use approximate aggregates for very large data if precision ok.

### Transformations & Aggregations — Example 6: `orderBy`

**What this is about:** Sort rows globally (caution: shuffle).

In [ ]:
df = df_products.orderBy(F.col("price").desc())
df.show()


**Explanation:**
`orderBy` triggers a shuffle; costly on large datasets.

**Optimization options:**
- Prefer sorting within partitions or limit before full sort.

### Transformations & Aggregations — Example 7: `join (inner)`

**What this is about:** Join sales with products.

In [ ]:
df = (df_sales.alias("s")
      .join(df_products.alias("p"), F.col("s.product_id") == F.col("p.product_id"), "inner")
      .select("s.customer_id", "s.product_id", "p.product", "s.qty", "p.price"))
df.show()


**Explanation:**
Inner join enriches sales rows with product info.

**Optimization options:**
- Broadcast small dimension tables to avoid shuffles (see hints).

### Transformations & Aggregations — Example 8: `join (left anti)`

**What this is about:** Find sales with missing product ref (data quality check).

In [ ]:
df_missing = (df_sales.alias("s")
              .join(df_products.alias("p"), F.col("s.product_id") == F.col("p.product_id"), "left_anti"))
df_missing.show()


**Explanation:**
`left_anti` returns left rows with **no** match; useful for audits.

**Optimization options:**
- Use `left_semi` to keep only matched left rows.

### Transformations & Aggregations — Example 9: `unionByName`

**What this is about:** Union DataFrames by matching column names.

In [ ]:
a = df_products.select("product_id","product","category","price")
b = df_products.select("product_id","product","category","price").withColumn("price", F.col("price")+10)
df = a.unionByName(b)
df.show()


**Explanation:**
`unionByName` is safer than `union` when schemas vary in order.

**Optimization options:**
- Use `allowMissingColumns=True` when schemas evolve.

### Transformations & Aggregations — Example 10: `explode`

**What this is about:** Explode array column into multiple rows.

In [ ]:
df_list = spark.createDataFrame([
    (1, ["k","m","l"]),
    (2, ["l","d"]),
], ["id","tags"])

df = df_list.select("id", F.explode("tags").alias("tag"))
df.show()


**Explanation:**
`explode` flattens arrays for per-element analysis.

**Optimization options:**
- Consider `posexplode` to keep positions if needed.

### Transformations & Aggregations — Example 11: `pivot`

**What this is about:** Pivot categories into columns with sums.

In [ ]:
df = (df_sales.alias("s")
      .join(df_products.alias("p"), "product_id")
      .groupBy("customer_id")
      .pivot("category")
      .agg(F.sum("qty").alias("qty_sum")))
df.show()


**Explanation:**
Pivot turns values into columns; great for wide summaries.

**Optimization options:**
- Limit pivot distinct values to avoid huge wide tables.

### Transformations & Aggregations — Example 12: `window functions`

**What this is about:** Use `row_number` over partitions.

In [ ]:
from pyspark.sql.window import Window
w = Window.partitionBy("customer_id").orderBy(F.col("sale_date").asc())
df = (df_sales
      .withColumn("rn", F.row_number().over(w)))
df.show()


**Explanation:**
Window functions compute metrics across row groups without collapsing rows.

**Optimization options:**
- Keep partitions balanced; sort keys selective.

### Transformations & Aggregations — Example 13: `na.fill / na.drop`

**What this is about:** Handle missing values.

In [ ]:
df = df_customers.na.fill({"city":"Unknown"}).na.drop(subset=["name"])
df.show()


**Explanation:**
`na` helpers treat nulls for robust processing.

**Optimization options:**
- Use column-wise strategies for numeric/string types.

### Transformations & Aggregations — Example 14: `cache / persist`

**What this is about:** Cache a reused DataFrame and then unpersist.

In [ ]:
df = (df_sales
      .join(df_products, "product_id")
      .cache())
print(df.count())  # materialize
df = df.unpersist()


**Explanation:**
Caching avoids recomputation for repeated actions.

**Optimization options:**
- Cache only reused, expensive intermediates; monitor storage.

### Transformations & Aggregations — Example 15: `UDF (user-defined function)`

**What this is about:** Apply a simple UDF (note: slower than built-ins).

In [ ]:
from pyspark.sql.functions import udf
@udf("string")
def city_tier(city):
    return "Tier-1" if city in ("Bangalore","Mumbai") else "Other"

df = df_customers.withColumn("city_tier", city_tier("city"))
df.show()


**Explanation:**
UDFs run row-wise and can be slower; prefer built-ins or pandas UDFs.

**Optimization options:**
- Use built-in functions or SQL expressions for vectorization.

## 3) Using Spark SQL for Querying — 15 Examples

### Spark SQL — Example 1: `createOrReplaceTempView`

**What this is about:** Register a DataFrame as a temporary view for SQL querying.

In [ ]:
df_products.createOrReplaceTempView("products")
df_customers.createOrReplaceTempView("customers")
df_sales.createOrReplaceTempView("sales")
spark.sql("SHOW TABLES").show(truncate=False)


**Explanation:**
Temp views live for the session; easy for pure-SQL workflows.

**Optimization options:**
- Use `createGlobalTempView` to span sessions within an app.

### Spark SQL — Example 2: `SELECT ... WHERE ... LIMIT`

**What this is about:** Basic SQL selection with filters and limits.

In [ ]:
spark.sql("""
SELECT product_id, product, price
FROM products
WHERE price > 1000
ORDER BY price DESC
LIMIT 5
""").show()


**Explanation:**
Familiar SQL makes quick analysis easy.

**Optimization options:**
- Add `LIMIT` during exploration to reduce shuffle/work.

### Spark SQL — Example 3: `GROUP BY + HAVING`

**What this is about:** Aggregate and filter groups by condition.

In [ ]:
spark.sql("""
SELECT category, COUNT(*) AS cnt, AVG(price) AS avg_price
FROM products
GROUP BY category
HAVING COUNT(*) >= 1
""").show()


**Explanation:**
`HAVING` filters aggregated groups.

**Optimization options:**
- Keep aggregations selective; bucket/partition for common group keys.

### Spark SQL — Example 4: `JOIN`

**What this is about:** Join sales with products.

In [ ]:
spark.sql("""
SELECT s.customer_id, s.product_id, p.product, s.qty, p.price
FROM sales s
JOIN products p ON s.product_id = p.product_id
""").show()


**Explanation:**
SQL join syntax mirrors DataFrame join.

**Optimization options:**
- Broadcast small tables using hints to avoid shuffles.

### Spark SQL — Example 5: `Window function: ROW_NUMBER`

**What this is about:** Rank rows per customer by sale_date.

In [ ]:
spark.sql("""
SELECT *, ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY sale_date) AS rn
FROM sales
""").show()


**Explanation:**
Window functions enable ranking and running metrics.

**Optimization options:**
- Partition on business keys; sort by selective columns.

### Spark SQL — Example 6: `CTE (WITH ...)`

**What this is about:** Use CTEs for readable multi-step SQL.

In [ ]:
spark.sql("""
WITH enriched AS (
  SELECT s.*, p.product, p.category, p.price
  FROM sales s JOIN products p ON s.product_id = p.product_id
)
SELECT customer_id, SUM(qty*price) AS revenue
FROM enriched
GROUP BY customer_id
""").show()


**Explanation:**
CTEs structure complex logic into blocks.

**Optimization options:**
- CTEs may be inlined; materialize via cache if reused.

### Spark SQL — Example 7: `Subquery`

**What this is about:** Use a subquery to filter by expensive products.

In [ ]:
spark.sql("""
SELECT *
FROM products
WHERE price > (SELECT AVG(price) FROM products)
""").show()


**Explanation:**
Subqueries encapsulate nested logic succinctly.

**Optimization options:**
- Beware of non-correlated vs correlated subqueries performance.

### Spark SQL — Example 8: `Aggregate with FILTER clause`

**What this is about:** Conditional aggregations with FILTER.

In [ ]:
spark.sql("""
SELECT
  COUNT(*) AS total,
  COUNT(*) FILTER (WHERE category='peripheral') AS peripheral_cnt
FROM products
""").show()


**Explanation:**
`FILTER` lets you compute conditional metrics in one pass.

**Optimization options:**
- Replace multiple scans with single-pass conditional aggs.

### Spark SQL — Example 9: `Date/time functions`

**What this is about:** Use `to_date` and `date_trunc`.

In [ ]:
spark.sql("""
SELECT to_date(sale_date) AS d, date_trunc('month', to_timestamp(sale_date)) AS month
FROM sales
""").show()


**Explanation:**
Built-in date/time funcs cover parsing and bucketing.

**Optimization options:**
- Align partitions by month/week for acceleration.

### Spark SQL — Example 10: `Regex extraction`

**What this is about:** Extract tokens using `regexp_extract`.

In [ ]:
spark.sql("""
SELECT product, regexp_extract(product, '(.*?)(?:$)', 1) AS token
FROM products
""").show()


**Explanation:**
`regexp_extract` parses text using regex groups.

**Optimization options:**
- Validate patterns; avoid overuse for performance.

### Spark SQL — Example 11: `CASE WHEN for conditional pivots`

**What this is about:** Compute category-wise sums with CASE.

In [ ]:
spark.sql("""
SELECT customer_id,
  SUM(CASE WHEN p.category='peripheral' THEN s.qty ELSE 0 END) AS qty_peripheral,
  SUM(CASE WHEN p.category='display' THEN s.qty ELSE 0 END) AS qty_display,
  SUM(CASE WHEN p.category='computer' THEN s.qty ELSE 0 END) AS qty_computer
FROM sales s JOIN products p ON s.product_id=p.product_id
GROUP BY customer_id
""").show()


**Explanation:**
Conditional aggregation achieves pivot-like summaries in SQL.

**Optimization options:**
- Prefer DF pivot API for many categories.

### Spark SQL — Example 12: `from_json / to_json`

**What this is about:** Parse/serialize JSON in SQL.

In [ ]:
spark.sql("""
SELECT product,
       to_json(named_struct('cat',category,'p',price)) AS as_json
FROM products
""").show()


**Explanation:**
`to_json`/`from_json` convert between struct and JSON.

**Optimization options:**
- Use schemas with `from_json` for robust parsing.

### Spark SQL — Example 13: `Broadcast hint`

**What this is about:** Use optimizer hint to broadcast a small table.

In [ ]:
spark.sql("""
SELECT /*+ BROADCAST(p) */ s.customer_id, s.product_id, p.product
FROM sales s JOIN products p ON s.product_id = p.product_id
""").show()


**Explanation:**
Broadcast avoids shuffle by sending small table to all workers.

**Optimization options:**
- Ensure the right side is small (fits driver/executor memory).

### Spark SQL — Example 14: `Global temp view`

**What this is about:** Create a global temp view usable across sessions in the same app.

In [ ]:
df_products.createGlobalTempView("g_products")
spark.sql("SELECT * FROM global_temp.g_products").show()


**Explanation:**
Global temp views are under the `global_temp` database.

**Optimization options:**
- Good for app-wide sharing without metastore tables.

### Spark SQL — Example 15: `EXPLAIN & config`

**What this is about:** Inspect plan and tweak SQL configs.

In [ ]:
spark.sql("EXPLAIN SELECT * FROM products WHERE price > 1000").show(truncate=False)
spark.sql("SET spark.sql.autoBroadcastJoinThreshold=10MB").show(truncate=False)


**Explanation:**
`EXPLAIN` shows logical/physical plans; configs steer the optimizer.

**Optimization options:**
- Only change critical configs deliberately, record them.

---
### Next Steps
- Swap demo data with your real datasets.
- Scale partitioning, caching, and joins based on data sizes.
- Add checkpoints, unit tests, and structured logging for production.
